In [1]:
# import Pkg; Pkg.add(["SimpleGraphs","JSON","TikzGraphs","TikzPictures","Graphs","DataStructures"])
import JSON
import TikzGraphs, TikzPictures
using SimpleGraphs
using Graphs
using DataStructures

In [95]:
#Partially based on https://github.com/scheinerman/SimplePosets.jl, but we're also permitting equality (their posets are
#all strict) and keeping track of known nonequality.
mutable struct LanguagePoset
  order::DirectedGraph{String}
  noteq::UndirectedGraph{String}
  function LanguagePoset(names::Vector{String})
    order = DirectedGraph{String}()
    noteq = UndirectedGraph{String}()
    for name = names
      add!(order, name)
      add!(order, name, name)
      add!(noteq, name)
    end
    new(order, noteq)
  end
end

import SimpleGraphs.has
has(P::LanguagePoset, x)  where T = has(P.order, x)
# Check if x≤y holds
has(P::LanguagePoset, x, y) where T = has(P.order, x, y)

function above(P::LanguagePoset, x::String)
    if !has(P,x)
        error("Class not in poset ", x)
    end
    return collect(P.order.N[x])
end
function below(P::LanguagePoset, x::String)
    if !has(P,x)
        error("Class not in poset ", x)
    end
    return collect(P.order.NN[x])
end
function equal(P::LanguagePoset, x::String)
    return intersect(above(P,x), below(P,x))
end

function add_order!(P::LanguagePoset, x::String, y::String)
    U = above(P,y)
    D = below(P,x)
  
    push!(U,y)
    push!(D,x)
    for u in U
        for d in D
            add!(P.order,d,u)
        end
    end
end

function add_equals!(P::LanguagePoset, x::String, y::String)
  add_order!(P,x,y)
  add_order!(P,y,x)
end

function add_notequal!(P::LanguagePoset, x::String, y::String)
  add!(P.noteq,x,y)
end

function is_equal(P::LanguagePoset, x::String, y::String)
  return has(P,x,y) && has(P,y,x)
end

function interval(P::LanguagePoset, x::String, y::String)
  A = Set(above(P,x))
  B = Set(below(P,y))
  C = Set(below(P,x))
  D = Set(above(P,y))
  return collect(setdiff(intersect(A,B),union(C,D)))
end

function equal_classes(P::LanguagePoset, x::String)
    return intersect(above(P, x),below(P, x))
end

#Get a list of all classes, but if there are several equal ones, only return one of those.
function nonequal_classes(P::LanguagePoset)
  return nothing
end

nonequal_classes (generic function with 1 method)

In [3]:
function prettyPrintClass(class)
  println(class["name"]," -- ",class["type"])
  println("  ",class["desc"])
  if "solver" in keys(class)
    println("  Def: ",class["solver"])
  end
  if "related" in keys(class)
    println("  See Also: ",join(class["related"],", "))
  end
end

function prettyPrint(db)
  for class = db["classes"]
    prettyPrintClass(class)
  end
end

prettyPrint (generic function with 1 method)

In [249]:
function make_language_poset(classlist=language_classes; verbose=false)
  #Assemble lattice of known inclusions and nonequalities
  class_lattice = LanguagePoset(collect(keys(classlist)))

  for (thmname, thm) = theorems
    content = thm["content"]
    if content == ""
      continue #no content implemented
    end
    if content[1] == '{'
      verbose && println("Skipping statement ",content)
      continue #depends on arguments, not implemented
    end
    #dead simple "parsing"
    for part = split(content,"&&")
      if part[1] == '{'
        verbose && println("Skipping statement ",part," from ",content)
        continue #depends on arguments, not implemented
      end
      rel = first.(String.(getfield.(collect(eachmatch(r"[⊂⊆⊃⊇≠=]", part)), :match)))
      if rel === nothing
        println("Unknown statement ",part)
        continue
      end
      @assert length(rel) == 1 content*" has multiple operators "*string(rel)
      rel = rel[1]
      lhs, rhs = String.(split(part, rel))
      @assert lhs in keys(classes) lhs, "Language "*lhs*" not known, in theorem "*thmname
      @assert rhs in keys(classes) rhs, "Language "*rhs*" not known, in theorem "*thmname
      if lhs ∉ keys(classlist) && lhs ∈ keys(classes)
        if verbose
          println("Skipping ",part," for being wrong language type")
        end
        continue
      end
      if rel == '⊃'
        rel = '⊂'
        lhs,rhs = rhs,lhs
      elseif rel == '⊇'
        rel = '⊆'
        lhs,rhs = rhs,lhs
      end
      #does this relation imply nonequality?
      is_neq = rel in ['⊂','≠']
      if rel == '⊂'
        rel = '⊆'
      end
      #left with ⊆, ≠, or =
      if rel == '⊆'
        verbose && println("Add ",lhs,"->",rhs)
        add_order!(class_lattice, lhs, rhs)
      elseif rel == '='
        verbose && println("Add ",lhs,"==",rhs)
        add_equals!(class_lattice, lhs, rhs)
      elseif rel == '≠'
        #do nothing, handled in is_neq
      else
        println("Strange relation ",rel)
      end
      if is_neq
        verbose && println("Add ",lhs,"!=",rhs)
        add_notequal!(class_lattice, lhs, rhs)
      end
    end
  end
  #Now go through class equalities and normalize.
  return class_lattice
end

function LangPoset_to_SimpleDiGraph(poset; verbose=false)
  g0 = poset.order
  vertlist = collect(g0.V)
  v_to_i = v -> findfirst(x->x==v, vertlist)
  edge_list = Edge{Int}[]
  for v1 = vertlist
    i1 = v_to_i(v1)
    for v2 = g0.N[v1]
      if v1 == v2
        continue #skip self-loops
      end
      if v1 ∈ g0.N[v2]
        continue #skip equality
      end
      i2 = v_to_i(v2)
      if length(interval(poset, v1, v2)) > 0
        verbose && println("skip ",v1," to ",v2)
        continue
      end
      verbose && println("add ",v1," to ",v2)
      push!(edge_list, Graphs.SimpleEdge{Int}(i1,i2))
    end
    end
  SimpleDiGraph(edge_list), vertlist
end

function export_for_web(class_lattice, langtype="Language")
    priority = 0 #TODO, for now all are equal priority
    
    #First filter out any classes that are equal to something else, except for the representatives of each
    equal_reps = Any[] #things to include
    equal_skips = Any[] #things to not
    for class_obj = db["classes"]
        if class_obj["type"] != langtype
            continue
        end
        class = class_obj["name"]
        
        #equal to something else?
        equals = equal_classes(class_lattice, class)
        if length(equals) > 1 #there's something else it's equal to
            canon_rep = intersect(equals, canonical_forms)
            if length(canon_rep) > 1
                error("Multiple representatives ",canon_rep," for equal classes ",equals)
            end
            if length(canon_rep) == 1
                canon_rep = canon_rep[1]
            else
                sort!(equals)
                canon_rep = equals[1]
                if canon_rep == class
                    println("Choosing ",class," as representative for ",equals)
                end
            end
            if canon_rep != class
                continue
            end
        end
        setdiff!(equals, [class]) #delete self from list of equals
        
        push!(equal_reps, class)
        append!(equal_skips, equals)
    end
    
    class_list = Any[]
    for class_obj = db["classes"]
        class = class_obj["name"]
        
        if class_obj["type"] != langtype #TODO: non-language classes
            continue
        end
        
        children = Any[]
        #only collect children if it's not skipped
        is_equal = class ∈ equal_skips
        if !is_equal
            
            for class2 = class_lattice.order.N[class] #loop over neighbors
                if class == class2 #skip over self
                    continue
                end
                if class2 ∈ equal_skips #skip over non-representative classes
                    continue
                end
                if length(interval(class_lattice, class, class2)) > 0 #skip over non-immediate children
                    continue
                end
                #TODO: add edge info on each child
                push!(children, class2)
            end
            
            equals = equal_classes(class_lattice, class)
            setdiff!(equals, [class])
        
        else
            equals = false
            
        end
        
        #TODO: "related", "notes", "properties"
        export_obj = Dict("name"=>class, "desc"=>class_obj["desc"], "children"=>children, "equals"=>equals)
        push!(class_list, export_obj)
    end
    
    class_list
end

export_for_web (generic function with 2 methods)

In [258]:
#useful characters: ≤ ≥ ⊂ ⊆ ⊃ ⊇ ≠ = ∩ ⟹ Ω Σ Π Δ

s_raw = read("classes.json", String)
s = replace(s_raw, r"\n[ \t]*//.*" => "") #strip comments
s = replace(s, "⊊" => "⊂") #normalize subsetnoteq
db = JSON.parse(s);

#Verify the database makes reasonable sense
@assert all("name" in keys(t) for t in db["problem types"])
@assert all("desc" in keys(t) for t in db["classes"])

@assert all("name" in keys(t) for t in db["classes"])
@assert all("type" in keys(t) for t in db["classes"])
@assert all("desc" in keys(t) for t in db["classes"])

@assert all("name" in keys(t) for t in db["conjectures"])
@assert all("content" in keys(t) for t in db["conjectures"])

@assert all("name" in keys(t) for t in db["theorems"])
@assert all("content" in keys(t) for t in db["theorems"])

problem_types = Dict((x->x["name"]=>x).(db["problem types"]))
classes = Dict((x->x["name"]=>x).(db["classes"]))
conjectures = Dict((x->x["name"]=>x).(db["conjectures"]))
theorems = Dict((x->x["name"]=>x).(db["theorems"]))

@assert all(t["type"] in keys(problem_types) for (n,t) in classes)

for (clsname, cls) = classes
  if "related" ∉ keys(cls)
    cls["related"] = Any[]
  end
  cls["related"] = Set{String}(cls["related"])
  for rlt = cls["related"]
    @assert (rlt ∈ keys(classes)) (clsname*" related to unknown class "*rlt)
  end
end

#Specifically classes that are languages (as opposed to e.g. promise problems)
language_classes = filter(x->x[2]["type"]=="Language", classes)
parametized_classes = filter(x->x[2]["type"]=="Parameterized Language", classes)

println("Initial checks passed")

Initial checks passed


In [259]:
canonical_forms = ["PSPACE","NC","L","NL","NC^0"]; #canonical names for classes that have equalities.

class_lattice = make_language_poset(verbose=false);
param_lattice = make_language_poset(parametized_classes; verbose=false);

el,vl = LangPoset_to_SimpleDiGraph(class_lattice; verbose=false)
tikzplot = TikzGraphs.plot(el, vl, node_style="draw", graph_options="nodes={draw,circle}")

TikzPictures.save(TikzPictures.TEX("testgraph"), tikzplot)

In [261]:
json_export = export_for_web(class_lattice)

open("public/langdata.json","w") do f
    JSON.print(f, json_export)
end

json_export = export_for_web(param_lattice, "Parameterized Language")

open("public/paramlangdata.json","w") do f
    JSON.print(f, json_export)
end

In [233]:
prettyPrint(db)

AC -- Language
  Unbounded Fanin Polylogarithmic-Depth Circuits. The class of decision problems solvable by a nonuniform family of Boolean circuits, with polynomial size, depth O(log^k(n)), and unbounded fanin, for some k. The gates allowed are AND, OR, and NOT. For a given k, we get AC^k, such as AC^1. The class AC^0 is when k=0 i.e. constant depth. Equal to NC and TC.
  See Also: NC, AC^0, AC^1, TC
AC^0 -- Language
  Unbounded Fanin Constant-Depth Circuits. An especially important subclass of AC, corresponding to constant-depth, unbounded-fanin, polynomial-size circuits with AND, OR, and NOT gates.
  See Also: NC, NC^0, AC^1, AC^0
AC^1 -- Language
  Unboudned Fanin Log-depth Circuits.
  See Also: AC, TC^1, NC^1, AC^0
ACC^0 -- Language
  AC0 With Arbitrary MOD Gates.
  See Also: AC^0, TC^0
ACKERMANN -- Language
  Ackermann function time. Problems solvable by a Turing machine in time A(O(n),O(n)), where A is the Ackermann function. Reachability in vector addition systems is complete fo

  See Also: ELEMENTARY, PR
ZPP -- Language
  Zero error Probabilistic Polynomial time. Problems with a probabilistic Turing machine with zero error and polynomial expected running time
  See Also: RP, BPP
Δ2 -- Language
  P^NP. The weakest class at the second level in the polynomial hierarchy.
  Def: P^NP
  See Also: Π2, Σ2, PH
Π2 -- Language
  Universal polynomial time, 2 alternations. coNP^NP, at the second level in the polynomial hierarchy.
  Def: coNP^NP
  See Also: Δ2, Σ2, PH
Σ2 -- Language
  Existential polynomial time, 2 alternations. NP^coNP, at the second level in the polynomial hierarchy.
  Def: NP^coNP
  See Also: Π2, Δ2, PH
PromiseBPP -- Promise Problem
  Promise Bounded error Probabilistic Polynomial time. Problems with a probabilistic polynomial-time Turing machine promised to have at least a 2/3 chance of a correct answer.
  See Also: BPP
TFNP -- Function Problem
  Total Function Nondeterministic Polynomial time. Relations that can be checked by deterministic polynomial 